In [ ]:
### Here is some of the code I used for my final year Chromsome-Condensation project - Taylor-Jai O'Connor ###

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from scipy.optimize import curve_fit

# Initial configurations for lengths per bead count
initial_lengths = {
    250: 1906,
    500: 2515,
    1000: 3318,
    2000: 4379,
    4000: 5778
}

# Initial guesses for xeq and tau
initial_guesses = {
    250: {'length': (1000, 100), 'width': (500, 25), 'depth': (275, 25)},
    500: {'length': (1750, 600), 'width': (750, 1000), 'depth': (400, 30)},
    1000: {'length': (1800, 1200), 'width': (1000, 400), 'depth': (600, 50)},
    2000: {'length': (2500, 1600), 'width': (850, 1600), 'depth': (400, 1600)},
    4000: {'length': (3000, 2000), 'width': (1100, 2000), 'depth': (500, 2000)},
}

def exp_decay(t, xeq, tau, x0):
    return xeq + (x0 - xeq) * np.exp(-t / tau)

def compute_average_dimensions(coords):
    cov_matrix = np.cov(coords.T)
    eigenvalues = np.linalg.eigvalsh(cov_matrix)
    eigenvalues = np.sort(eigenvalues)[::-1]
    return 2 * np.sqrt(eigenvalues)  # length, width, depth, in descending order

def fit_decay(time, values, bead_count, dimension):
    x0 = initial_lengths[bead_count]
    xeq_guess, tau_guess = initial_guesses[bead_count][dimension]

    def decay_model(t, xeq, tau):
        return exp_decay(t, xeq, tau, x0)

    try:
        popt, _ = curve_fit(
            decay_model, np.array(time), np.array(values),
            p0=[xeq_guess, tau_guess],
            bounds=([0, 1], [x0, np.inf]),
            maxfev=20000
        )
        return x0, popt[0], popt[1]
    except Exception as e:
        print(f"Fit error for {dimension}, {bead_count} beads: {e}")
        return x0, np.nan, np.nan

def process_replicate(file_path):
    dims = {}
    try:
        df = pd.read_csv(file_path, delim_whitespace=True, usecols=['time(s)', 'Type(bead)', 'x(nm)', 'y(nm)', 'z(nm)'])
        df = df[df['Type(bead)'] == 'N']
        time_points = sorted(df['time(s)'].unique())

        dims['time'] = []
        dims['length'] = []
        dims['width'] = []
        dims['depth'] = []

        for t in time_points:
            coords = df[df['time(s)'] == t][['x(nm)', 'y(nm)', 'z(nm)']].values
            length, width, depth = compute_average_dimensions(coords)
            dims['time'].append(t)
            dims['length'].append(length)
            dims['width'].append(width)
            dims['depth'].append(depth)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
    return pd.DataFrame(dims)

def plot_fitted_curve(time, values, xeq, tau, bead_count, dimension, output_path):
    x0 = initial_lengths[bead_count]
    plt.figure(figsize=(8, 6))
    plt.plot(time, values, label='Mean Data')
    if not np.isnan(xeq) and not np.isnan(tau):
        fitted = exp_decay(np.array(time), xeq, tau, x0)
        plt.plot(time, fitted, '--', label=f'Exp Fit\nxeq={xeq:.1f}, tau={tau:.1f}')
    plt.xlabel('Time (s)')
    plt.ylabel(f'{dimension.capitalize()} (nm)')
    plt.title(f'Average {dimension.capitalize()} vs Time\n{bead_count} Beads')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

def main():
    base_dir = r""
    output_dir = r""
    os.makedirs(output_dir, exist_ok=True)

    bead_counts = [250, 500, 1000, 2000, 4000]
    dimensions = ['length', 'width', 'depth']
    summary_rows = []

    for bead_count in bead_counts:
        replicate_paths = [
            os.path.join(base_dir, str(bead_count), f'replicate_{i}', '_msd_bead_coord.txt')
            for i in range(1, 11)
        ]

        time_list = None
        dim_accum = {dim: [] for dim in dimensions}

        for rep_idx, file_path in enumerate(replicate_paths, 1):
            if not os.path.isfile(file_path):
                print(f"Missing: {file_path}")
                continue

            df = process_replicate(file_path)
            if df.empty:
                continue

            if time_list is None:
                time_list = df['time'].values

            for dim in dimensions:
                dim_accum[dim].append(df[dim].values)

        if time_list is None:
            print(f"No data found for {bead_count} beads")
            continue

        time_array = np.array(time_list)
        for dim in dimensions:
            dim_matrix = np.array(dim_accum[dim])
            mean_values = np.mean(dim_matrix, axis=0)
            x0, xeq, tau = fit_decay(time_array, mean_values, bead_count, dim)

            summary_rows.append({
                'bead_count': bead_count,
                'dimension': dim,
                'x0': x0,
                'xeq': xeq,
                'tau': tau
            })

            plot_path = os.path.join(output_dir, f'{bead_count}_{dim}_fit.png')
            plot_fitted_curve(time_array, mean_values, xeq, tau, bead_count, dim, plot_path)

    # Summary CSV file 
    summary_df = pd.DataFrame(summary_rows)
    summary_csv_path = os.path.join(output_dir, 'summary_fit_results.csv')
    summary_df.to_csv(summary_csv_path, index=False)

main()
